### 安装项目所需的第三方库

In [ ]:
!pip install parl -i https://pypi.tuna.tsinghua.edu.cn/simple
!pip install visualdl -i https://mirror.baidu.com/pypi/simple
!pip install gym==0.18.0 -i https://pypi.tuna.tsinghua.edu.cn/simple
!pip install numpy -i https://pypi.tuna.tsinghua.edu.cn/simple

### 项目文件介绍

agent.py  -----> 强化学习智能体（包含Sarsa算法）<br>
gridworld.py -----> 游戏环境（包含悬崖环境）<br>
train.py -----> 训练强化学习智能体 <br>


### Q表格示例
![](https://ai-studio-static-online.cdn.bcebos.com/ef57d8b92cb24cd4ae651892c99a00690a199689e06b4ebb9f742c313fab7f85)


### Sarsa 算法

![](https://ai-studio-static-online.cdn.bcebos.com/77ca24ade3ef41f5a853fb64b6224e37590453181f4547b0859176231e4263dd)

### Sarsa 算法 初始化
``` python
    def __init__(self,
                     obs_n,
                     act_n,
                     learning_rate=0.01,
                     gamma=0.9,
                     e_greed=0.1):
            self.act_n = act_n  # 动作维度，有几个动作可选
            self.lr = learning_rate  # 学习率
            self.gamma = gamma  # reward的衰减率
            self.epsilon = e_greed  # 按一定概率随机选动作
            self.Q = np.zeros((obs_n, act_n))  # 建立Q表格
```
其中：
学习率 $\alpha$ 默认为 0.01
reward的衰减率（折扣因子）$\gamma$ 默认为 0.9
探索率 $\varepsilon$ 默认为 0.1 ，即10%的概率进行探索（随机动作）

### Sarsa 算法 带探索的动作采样
```Python
    # 根据输入观察值，采样输出的动作值，带探索
            def sample(self, obs):
                if np.random.uniform(0, 1) < (1.0 - self.epsilon):  #根据table的Q值选动作
                    action = self.predict(obs)
                else:
                    action = np.random.choice(self.act_n)  #有一定概率随机探索选取一个动作
                return action
```
其中：
使用 90% 的概率，从Q表格中选取当前 **状态$obs$** 的Q值最大的动作 （利用）
使用 10% 的概率，从所有可选择的动作中，随机选择动作 （探索）

### Sarsa 算法 从策略（Q表格）中输出最优动作
```Python
    # 根据输入观察值，预测输出的动作值
    def predict(self, obs):
        Q_list = self.Q[obs, :]  # 获取状态obs下所有动作的Q值

        maxQ = np.max(Q_list)  # 获取最大的Q值（maxQ）

        action_list = np.where(Q_list == maxQ)[0]
        # 获取最大Q值的索引位置（最优动作）
        # maxQ可能对应多个action，意味着最优动作可能不只一个

        action = np.random.choice(action_list)
        # 选取最优动作，如果在最优动作不只一个的情况下，从中随机选择一个

        return action
```


### Sarsa 算法 策略更新
```Python
    # 学习方法，也就是更新Q-table的方法
        def learn(self, obs, action, reward, next_obs, next_action, done):
            """ on-policy
                obs: 交互前的obs, s_t
                action: 本次交互选择的action, a_t
                reward: 本次动作获得的奖励r
                next_obs: 本次交互后的obs, s_t+1
                next_action: 根据当前Q表格, 针对next_obs会选择的动作, a_t+1
                done: episode是否结束
            """
            predict_Q = self.Q[obs, action]
            if done:
                target_Q = reward  # 没有下一个状态了
            else:
                target_Q = reward + self.gamma * self.Q[next_obs, next_action]  # Sarsa

            self.Q[obs, action] += self.lr * (target_Q - predict_Q)  # 修正q
```
其中：
> target_Q = reward + self.gamma * self.Q[next_obs, next_action]

公式为：$Q_{target}=R+\gamma Q(S_{t+1}, A_{t+1})$

> self.Q[obs, action] += self.lr * (target_Q - predict_Q)

公式为：$Q(S_t,A_t)=Q(S_t,A_t)+\alpha (Q_{target}-Q(S_t,A_t))$